# Day 21: Generated Kubernetes Manifests

Use an LLM to generate ready-to-apply Kubernetes Deployment and Service YAML from a simple service description.

**Steps:**
1. Define service requirements (name, image, ports, replicas, environment variables).
2. Prepare a prompt for the LLM to generate Kubernetes manifests.
3. Call the LLM (Ollama, Llama3, GPT-3.5, etc.) for YAML output.
4. Save the generated YAML to files for direct application.
5. (Optional) Validate YAML syntax and structure.

In [16]:
# Install required libraries
!pip install requests --quiet

In [17]:
# Import libraries
import requests
import yaml

In [18]:
# Define service requirements
service_description = {
    'name': 'webapp',
    'image': 'nginx:1.25',
    'replicas': 3,
    'ports': [80],
    'env': {
        'ENVIRONMENT': 'production',
        'API_URL': 'https://api.example.com'
    }
}
print('Service description:', service_description)

Service description: {'name': 'webapp', 'image': 'nginx:1.25', 'replicas': 3, 'ports': [80], 'env': {'ENVIRONMENT': 'production', 'API_URL': 'https://api.example.com'}}


In [19]:
# Prepare prompt for LLM to generate Kubernetes manifests
prompt = (
    "You are a Kubernetes expert. Given the following service description, generate a ready-to-apply Kubernetes Deployment and Service YAML. "
    "The YAML must be valid, production-ready, and include all required fields.\n\n"
    f"Service description:\n{service_description}\n\n"
    "Output only the YAML for both resources, separated by '---'."
)

In [20]:
# Call Ollama API for YAML output
OLLAMA_URL = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "llama3"

payload = {
    "model": OLLAMA_MODEL,
    "prompt": prompt,
    "stream": False
}
response = requests.post(OLLAMA_URL, json=payload)
yaml_output = response.json().get("response", "No YAML received.")

# Print and save YAML output
print(yaml_output)
with open("deployment_and_service.yaml", "w") as f:
    f.write(yaml_output)
print("YAML saved to deployment_and_service.yaml")

Here are the generated Kubernetes Deployment and Service YAML:

```
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: webapp
spec:
  replicas: 3
  selector:
    matchLabels:
      app: webapp
  template:
    metadata:
      labels:
        app: webapp
    spec:
      containers:
      - image: nginx:1.25
        name: webapp
        env:
        - name: ENVIRONMENT
          value: production
        - name: API_URL
          value: https://api.example.com
        ports:
        - containerPort: 80

---
apiVersion: v1
kind: Service
metadata:
  name: webapp
spec:
  selector:
    app: webapp
  ports:
  - name: http
    port: 80
    targetPort: 80
  type: LoadBalancer
```

Note that I've assumed a LoadBalancer service type to expose the service to the world. If you want to use a different service type (e.g., ClusterIP, NodePort), simply modify the `type` field accordingly.
YAML saved to deployment_and_service.yaml


In [23]:
# (Optional) Validate YAML syntax
# Extract only YAML blocks (between 'apiVersion:' or 'kind:' and '---' or end)
def extract_yaml_blocks(yaml_text):
    lines = yaml_text.splitlines()
    blocks = []
    current_block = []
    in_yaml = False
    for line in lines:
        # Remove code fences and skip lines with backticks
        if line.strip().startswith('```') or line.strip().endswith('```') or '`' in line:
            continue
        # Start of YAML block
        if line.strip().startswith('apiVersion:') or line.strip().startswith('kind:'):
            if current_block:
                blocks.append('\n'.join(current_block))
                current_block = []
            in_yaml = True
        # End of YAML block (separator)
        if in_yaml and line.strip() == '---':
            blocks.append('\n'.join(current_block))
            current_block = []
            in_yaml = False
            continue
        # Stop at the first non-YAML block after YAML (if LLM adds explanations at the end)
        if in_yaml and (line.strip().startswith('*') or line.strip().startswith('#')):
            break
        if in_yaml:
            current_block.append(line)
    # Add last block if any
    if current_block:
        blocks.append('\n'.join(current_block))
    return '\n---\n'.join(blocks)

yaml_only = extract_yaml_blocks(yaml_output)
with open("deployment_and_service.yaml", "w") as f:
    f.write(yaml_only)
print("YAML (only) saved to deployment_and_service.yaml")

# Validate YAML syntax
try:
    docs = list(yaml.safe_load_all(yaml_only))
    print("YAML validation successful. Contains:", len(docs), "resources.")
except Exception as e:
    print("YAML validation failed:", e)

YAML (only) saved to deployment_and_service.yaml
YAML validation successful. Contains: 4 resources.
